# Part 1: Altitude PID in Simulation

In this part of the project, you will be implementing a PID controller for a simulated drone that can only move in one dimension, the vertical dimension. You can control the speed the motors spin on the drone, which sets the thrust being generated by the propellers. In this system, the process variable is the drone's altitude, the setpoint is the desired altitude, and the error is the distance in meters between the setpoint and the drone's altitude. The output of the control function is a [PWM (pulse-width modulation)](https://en.wikipedia.org/wiki/Pulse-width_modulation)
value between 1100 and 1900, which is sent to the flight controller to
set the drone's throttle.

You should implement the discretized version of the PID control function in `student_pid_class.py`, inside the folder `packages/project-pid-implementation-<yourGitHubUsername>`:

$$u(t) = K_pe(t_k) + K_i\sum_{i=0}^k e(t_i)\Delta t + K_d\frac{e(t_k)-e(t_{k-1})}{\Delta t} + K$$

$$K_p, K_i, K_d, K = Constants \; and \; Offset \; Term$$
$$e(t_k) = Error \; at \; Time \; t_k$$
$$\Delta t = Time \; Elapsed \; from \; Previous \; Iteration$$

Notice that there is an extra offset term $K$ added to the control function. This is the base PWM value/throttle command before the three control terms are applied to correct the error in the system.

Write brief answers to all exercises in `answers_pid.md`.   



## Problem 1:  Implement an Idealized PID

**Exercises**

  1. Implement the `step` method to return the constant $K$. At what value of
  $K$ does the drone takeoff? Set $K$ to 1300 for the remainder of the questions.
  2. Implement the P term. What happens when $K_p$ is 50? 500? 5000?  
  3. Implement the D term. Set $K_p$ to zero. What happens when $K_d$ is 50? 500? 5000?  
  4. Now tune $K_p$ and $K_d$ so that the drone comes to a steady hover. Describe the trade-off as you change the ratio of $K_p$ to $K_d$.
  5. Implement the I term and observe the difference between PD and PID control. What role does the I term play in this system? What happens when $K_p$ and $K_d$ are set to zero?  
  6. Implement the `reset` method and test its behavior. If implemented incorrectly, what problems can you anticipate reset causing?  
  7. Finally, tune the constants in your PID controller to the best of your abilities. When the setpoint is moving, the drone should chase the setpoint very closely. When the setpoint is still, the drone should converge exactly at the setpoint and not oscillate. Report your tuning values.


# Tuning the gains of your PID

To tune your PID, set the parameters ($K_p, K_i, K_d, K$) with the sliders below (you can also change them manually in `z_pid.yaml`).  

The PID class in `student_pid_class.py` will automatically be used to control the simulated drone.

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import sys

# TODO: modify <yourGitHubName> to make this the path to your exercise folder
EXERCISE_DIRECTORY = "../packages/project-pid-implementation-Tuxliri/"
GAINS_PATH = EXERCISE_DIRECTORY + "z_pid.yaml"

sys.path.append(EXERCISE_DIRECTORY)

import matplotlib.pyplot as plt
import yaml
import ipywidgets as widgets
from utils.writer import load_gains, update_gains

from drone_sim import VerticalDrone

from student_pid_class import PID

%matplotlib widget


kp_widget = widgets.FloatSlider(value=1000.0,description='Kp')
kd_widget = widgets.FloatSlider(description='Kd')
ki_widget = widgets.FloatSlider(description='Ki')

k_widget = widgets.BoundedFloatText(value=1000,description='K')
drag_coeff=widgets.BoundedFloatText(value=0.2,description='Drag')
latency=widgets.BoundedIntText(value=6,description='Latency')
noise=widgets.BoundedFloatText(value=10.0,description='Noise')
height_setpoint=widgets.FloatSlider(value=0.5,min=0.0,max=1.0,description="Setpoint")

ui = widgets.HBox(
    [
        widgets.VBox([kp_widget,kd_widget,ki_widget,k_widget]),
        widgets.VBox(
            [   
                height_setpoint,
                drag_coeff,
                latency,
                noise,
            ]
        )
    ]
    )


def update(height_setpoint,drag_coeff,latency,noise,kp,kd,ki,k):
    update_gains(kp=kp,kd=kd,ki=ki,k=k,filepath=GAINS_PATH)

    pid_gains = load_gains(GAINS_PATH)

    my_pid_instance = PID(
    kp=pid_gains['Kp'],
    kd=pid_gains['Kd'],
    ki=pid_gains['Ki'],
    k=pid_gains['K'],
    )

    sim = VerticalDrone(
                    pid=my_pid_instance,
                    step_size=10,
                    drag_coeff=drag_coeff,
                    latency=latency,
                    sensor_noise=noise,
                    )

    sim.update_setpoint(height=height_setpoint)
    sim.simulate()
    sim.plot_step_response()

out = widgets.interactive_output(
    update,
    {
        'height_setpoint': height_setpoint,
        'drag_coeff' : drag_coeff,
        'latency' : latency,
        'noise' : noise,
        'kp' : kp_widget,
        'kd' : kd_widget,
        'ki' : ki_widget,
        'k'  : k_widget
    }
    )


In [18]:
# Execute this cell to show the tuner
display(ui,out)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "Canvas(toolbar=Toolbar(toolitems=[('Ho…


## Problem 2: Tuning a PID with Latency

Now, we introduce latency!  Now add a latency of `6` steps in the widget below to introduce 24 milliseconds of latency (six steps of latency running at 25 hz).  

**Exercises**

  1. Tune the constants in your PID controller to the best of your abilities. The drone should chase the setpoint very closely, but will converge more slowly when the setpoint is still. Report your tuning values.  
  2. Compare your tuning values to the values you obtained in problem 1.  
  3. Explain the effect of latency on each control term.  


In [19]:
# Execute this cell to show the tuner
display(ui,out)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "Canvas(toolbar=Toolbar(toolitems=[('Ho…


## Problem 3: Tuning a PID with Latency, Noise, and Drag

In the most realistic mode, you will tune a controller with latency,
noise, and a drag coefficient.  You can do this with the command line
arguments `python sim.py -l 3 -n 0.5 -d 0.02` to be most realistic to
real-world flight.

**Exercises**

  1. Tune with these arguments to be as good as possible. Report your tuning values.  
  2. Compare your tuning values to the values from problems 1 and 2.  

Run `python sim.py -h` to see the other simulator parameters. We
encourage you to experiment with those and observe their
effects on your controller.

**After you finish this part of the project, make sure that you push the final versions of the files that you modified to your Github repo.**

In [5]:
# Execute this cell to show the tuner
display(ui,out)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "Canvas(toolbar=Toolbar(toolitems=[('Ho…